## Distinguishing Failed Experiments with Rubicon 
Modeling runs can fail for a number of reasons. When logging with rubicon_ml, a failed run may result in an empty or incomplete experiment. In this example, we'll walk through how to handle such experiments.

First lets simulate the problem. To do this we'll create an estimator that will fail on it's `fit()` 30% of the time.  We will consider any experiment that has a `none` value for `self.state`.

In [1]:
from sklearn.base import BaseEstimator
import random

class BadEstimator(BaseEstimator):
    def __init__(self, state=None):
        super().__init__()
        self.state=None
        self.knn = KNeighborsClassifier(n_neighbors=2)
    def fit(self, X, y):
        self.knn.fit(X, y)
        output=random.random()
        if output<.3:
            output=None
        self.state=output
    def score(self, X):
        knn_score = self.knn.score(X)
        return knn_score

Now let's attempt to create a `rubicon_ml.sklearn` pipeline with this failing estimator and attempt to `fit` the pipeline twenty times. 

In [2]:
from rubicon_ml.sklearn import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from rubicon_ml import Rubicon

random.seed(18)

rubicon = Rubicon(
    persistence="memory",
)
project = rubicon.get_or_create_project(name="Failed Experiments")

X = [[1], [1], [1], [1]]
y = [1, 1, 1, 1]
pipe=make_pipeline(project, SimpleImputer(strategy="mean"),BadEstimator())
for _ in range(20):
    pipe.fit(X,y)

Now, tag any experiment that doesn't have a valid `badestimator__state` parameter for failures with `exp.add_tags(["failed"])`.

In [3]:
for exp in project.experiments():
    for p in exp.parameters():
        if (p.name=="badestimator__state" and p.value==None):
            exp.add_tags(["failed"])

Finally, we can now retrieve all our failed experiments by passing the `tags=["failed"]` to `project.experiments()`.

In [4]:
for exp in project.experiments(tags=["failed"]):
    print(exp)

Experiment(project_name='Failed Experiment', id='55f2bac0-a0c8-473b-a5f0-59219071b248', name='RubiconPipeline experiment', description=None, model_name=None, branch_name=None, commit_hash=None, training_metadata=None, tags=['failed'], created_at=datetime.datetime(2022, 5, 9, 18, 54, 12, 512110))
Experiment(project_name='Failed Experiment', id='cbf281bd-f110-4a46-a4e0-1bcf4ed7d383', name='RubiconPipeline experiment', description=None, model_name=None, branch_name=None, commit_hash=None, training_metadata=None, tags=['failed'], created_at=datetime.datetime(2022, 5, 9, 18, 54, 12, 516871))
Experiment(project_name='Failed Experiment', id='766df32f-6b1e-4f5e-b22e-868e7f5e4baf', name='RubiconPipeline experiment', description=None, model_name=None, branch_name=None, commit_hash=None, training_metadata=None, tags=['failed'], created_at=datetime.datetime(2022, 5, 9, 18, 54, 12, 525243))
Experiment(project_name='Failed Experiment', id='a8d2bbef-ab06-450b-b73c-acdfaaaf6e9e', name='RubiconPipeline